In [1]:
import os
import io
from copy import deepcopy
from pathlib import Path
from PIL import Image as PILImage
import cv2
import numpy as np
import re

from IPython.display import display, Audio
from ipywidgets import widgets, interact

from some_utils.photo_utils import exif_rotate, get_photo, get_centermost_detections
from ocr.google_api import save_cuts_for_detection, detect_text, draw_ocr
import ocr.ocr_utils as ocr_fcns

home_dir = os.getcwd()
# print(home_dir)
# print("Welcome!")

Welcome!


In [2]:
# imports from yolo directory

# TODO: make it simple, this is workaround
os.chdir(Path(home_dir, "yolov5"))
from model import Model
os.chdir(home_dir)
#print(home_dir)

In [3]:
# pricetag model
yolo_price_weight_path = str(Path("yolov5", "weights", "pricetag_small.pt"))
yolo_price_model = Model(yolo_price_weight_path, device="cpu")  # ~ 0.2s on CPU 

# product name and price model
yolo_price_weight_path = str(Path("yolov5", "weights", "product_name_and_price_small_albert_320.pt"))
yolo_product_name_model = Model(yolo_price_weight_path, device="cpu")  # ~ 0.2s on CPU 
yolo_product_name_model.img_size = 320

#print("Models loaded")

Models loaded


In [4]:
### run testing photos in cycle, go through all and draw results 
from copy import deepcopy
#import unidecode

# a list where ocr-read product names and prices will be stored
# for all centermost pricetags in dir_in
taglist = list()

dir_in = "./test_images/"

# dir_out = "./test_images_out_v2/"

# if not os.path.exists(dir_out):
#     os.mkdir(dir_out)

for i in os.listdir(dir_in):
    if not i.endswith("jpg"):
        continue
    try:
        im_name = os.path.join(dir_in, i)

        im_file = exif_rotate(PILImage.open(im_name))
        
        # detected pricetags
        det_pricetags = yolo_price_model.detect([np.array(im_file)],  mode="return")

        # selected pricetags cropped from the entire photo
        sel_pricetags = get_centermost_detections(im_file,
                                                  det_pricetags,
                                                  n=1,
                                                  mode="cut",
                                                  bigger_box_by_pct=0.0
                                                  )

        # detected product name and price bounding boxes (from a single pricetag)
        det_product_name = yolo_product_name_model.detect([sel_pricetags[0]],  mode="return")
        
        # selected name and price bounding boxes (at most 3 assumed in a photo)
        # TODO: without using get_centermost_detec
        sel_pricetag_parts = get_centermost_detections(sel_pricetags[0],
                                                       det_product_name,
                                                       n=4,
                                                       mode="cut",
                                                       bigger_box_by_pct=0.0,
                                                       centerpoint=np.array([0,0]),
                                                       )
        if len(sel_pricetags)>2:
            x = sel_pricetags
            break
        # paths to images cut to the bounding boxes of price and product name
        cuts_path = save_cuts_for_detection(sel_pricetag_parts)
        
        # outputs of the OCR for each the image cuts (cuts with product names or prices)
        cuts_ocr = [detect_text(path) for path in cuts_path]
        
#         # find price
#         price = ocr_fcns.find_detection_using_yolo(
#             cuts_ocr[0],
#             det_product_name,
#             yolo_product_name_model.names,
#             "price",
#             what_to_ignore=None,
#             n_to_return=1
#         )
#         # fcn returns text and bbox, print only text
#         result_price = " ".join(price[0])

#         # find product_name
#         product_name = ocr_fcns.find_detection_using_yolo(
#             google_detections=cuts_ocr[0],
#             yolo_detections=det_product_name,
#             yolo_classes=yolo_product_name_model.names,
#             what_to_find="product_name",
#             what_to_ignore="price",
#             n_to_return=3
#         )
        # fcn returns text and bbox, print only text
#         result_product_name = " ".join(product_name[0])

#         text = f"{result_product_name}: {result_price}"
        
        # append ocr info in the form [[product name], [price]]
        taglist.append([cuts_ocr[0][1], cuts_ocr[1][1]])
#         out = cv2.putText(
#             deepcopy(sel_pricetags[0]),
#             unidecode.unidecode(text), 
#             (10, int(sel_pricetags[0].shape[0]*0.95)), 
#             cv2.FONT_HERSHEY_SIMPLEX, 
#             1,
#             (0,255,0),
#             2
#         )
        # assumes only one output photo per per input photo
#         PILImage.fromarray(out).save(os.path.join(dir_out,i))
    
    
    except Exception as e:
        print(f"Error {i}: {e}")
    

In [5]:
# import json

# with open("names_prices_separately.json") as f:
#     tagdict = json.load(f)

In [6]:
# tagdict = dict()
# for tag in taglist:
#     try:
#         if len(tag[1]) == 0:
#             tag[1] = "empty"
#         if len(tag[0]) == 0:
#             tag[0] = "empty"
#         tagdict[tag[0][0]] = tag[1]
#     except:
#         tagdict[tag[0]] = tag[1]

In [7]:
# from copy import deepcopy
# tagdict_work = deepcopy(tagdict)

In [8]:
# x = ['1990\nKE\n', '1990', 'KE']
# x = ['00\n239°\nKE\n', '00', '239°', 'KE']
# x = ['1990%\nKE\n','1990','KE']

# # a bunch of regex heuristics:
# # 1. complete results:
# # if we find 3 boxes (len(x)==4), they are likely price in crowns, haller and Kč

# # 2. fixing crowns
# # sometimes Kč is corrupted => replacing K[anything] with Kč if in the last bbox
# # also, we delete the info about crowns, redundant


# # 3. fixing swapped haller and crowns
# # haller and crowns sometimes swapped order -> if 9[something] or 00 in crowns
# # AND the same pattern not in haller then swap order
# heur_2_string = r"^\s*9\d$|^\s*00\s*$"

# # 4. fixing merged haller and crowns
# # if there is a 4+ digit number in the price and it ends with 9[something] or 0[something]
# # split to two two digit numbers
# heur_3_string = r"\d{2,}(?=9\d)|\d{2,}(?=0\d)"

# # 5. cleaning price strings (only numbers permissible => Albert does not use commas)

# # heuristic 1
# if len(x) == 4:
#     # heuristic 2
#     if re.match(heur_2_string, x[1]) and not re.match(heur_2_string, x[2]):
#         swap = x[1]
#         x[1] = x[2]
#         x[2] = swap

# # heuristic 2
# x[-1] = re.sub(r"K.$","Kč",x[-1])
# if re.match(r"Kč",x[-1]):
#     del x[-1]

# # heuristic 3
# if len(x) < 4:
#     # split off haller
#     if re.match(heur_3_string, x[-1]):
#         x.append(x[-1][-2:])
#         x[-2] = x[-2][:-2]
#    # elif re.match(heur_3_string, x[-2]) and re.match():

# #heuristic 5
# for i in range(1,len(x)):
#     x[i] = re.sub(r"\D","",x[i])

In [9]:
# for key in tagdict_work:
#     if tagdict_work[key] != "empty":
#         # heuristic 1
#         if len(tagdict_work[key]) == 4:
#             # heuristic 2
#             if re.match(heur_2_string, tagdict_work[key][1]) and not re.match(heur_2_string, tagdict_work[key][2]):
#                 swap = tagdict_work[key][1]
#                 tagdict_work[key][1] = tagdict_work[key][2]
#                 tagdict_work[key][2] = swap

#         # heuristic 2
#         tagdict_work[key][-1] = re.sub(r"K.$","Kč", tagdict_work[key][-1])
#         if re.match(r"Kč", tagdict_work[key][-1]):
#             del tagdict_work[key][-1]

#         # heuristic 3
#         if len(tagdict_work[key]) < 4:
#             # split off haller
#             if re.match(heur_3_string, tagdict_work[key][-1]):
#                 tagdict_work[key].append(tagdict_work[key][-1][-2:])
#                 tagdict_work[key][-2] = tagdict_work[key][-2][:-2]
#            # elif re.match(heur_3_string, x[-2]) and re.match():

#         #heuristic 5
#         for i in range(1,len(tagdict_work[key])):
#             tagdict_work[key][i] = re.sub(r"\D","",tagdict_work[key][i])

In [10]:
# len(tagdict_work)

In [11]:
# with open("cleaned_names_prices.json",'w+') as f:
#     json.dump(tagdict_work,f, indent=4)

In [12]:
# ### TESTING CELL, COMMENT FOR VOILA USE
# # 20200826_090817 
# # 20200714_174205
# # 20200714_174224+
# im_file = exif_rotate(PILImage.open("./test_images/20200714_174224.jpg"))
# print(np.array(im_file)
#       .shape)

# # det_pricetags = yolo_price_model.detect([np.array(im_file)],  mode="draw")
# # display(PILImage.fromarray(det_pricetags))

# det_pricetags = yolo_price_model.detect([np.array(im_file)],  mode="return")
# print(f"{len(det_pricetags['cls'])} pricetags found")

# det_pricetags["rois"]

# sel_pricetags = get_centermost_detections(im_file, det_pricetags, n=1, mode="cut", bigger_box_by_pct=0.01)
# display(PILImage.fromarray(sel_pricetags[0]))

In [13]:
# det_product_name = yolo_product_name_model.detect([sel_pricetags[0]],  mode="return")


In [14]:
# ### TESTING CELL, COMMENT FOR VOILA USE

# # detect price and product name with yolo model, only the first pricetag
# det_product_name = yolo_product_name_model.detect([sel_pricetags[0]],  mode="return")
# # display(PILImage.fromarray(det_product_name))
# print(det_product_name)
# # print(yolo_product_name_model.names)

In [15]:
# ### TESTING CELL, COMMENT FOR VOILA USE

# # find price
# price = ocr_fcns.find_detection_using_yolo(
#     cuts_ocr[0],
#     det_product_name,
#     yolo_product_name_model.names,
#     "price",
#     what_to_ignore=None,
#     n_to_return=2
# )
# # fcn returns text and bbox, print only text
# print(" ".join(price[0]))

# # find product_name
# product_name = ocr_fcns.find_detection_using_yolo(
#     google_detections=cuts_ocr[0],
#     yolo_detections=det_product_name,
#     yolo_classes=yolo_product_name_model.names,
#     what_to_find="product_name",
#     what_to_ignore="price",
#     n_to_return=2
# )
# # fcn returns text and bbox, print only text
# print(" ".join(product_name[0]))

In [16]:
# a bunch of regex heuristics:
# 1. complete results:
# if we find 3 boxes (len(x)==4), they are likely price in crowns, haller and Kč

# 2. fixing crowns
# sometimes Kč is corrupted => replacing K[anything] with Kč if in the last bbox
# also, we delete the info about crowns, redundant


# 3. fixing swapped haller and crowns
# haller and crowns sometimes swapped order -> if 9[something] or 00 in crowns
# AND the same pattern not in haller then swap order
heur_2_string = r"^\s*9\d$|^\s*00\s*$"

# 4. fixing merged haller and crowns
# if there is a 4+ digit number in the price and it ends with 9[something] or 0[something]
# split to two two digit numbers
heur_3_string = r"\d{2,}(?=9\d)|\d{2,}(?=0\d)"

# 5. cleaning price strings (only numbers permissible => Albert does not use commas)

In [ ]:
print("Personal Shelf Inspector")

In [23]:
def callback(value):
    if btn_upload.data:

        #im_name = "test_images/20200714_174205.jpg"
        #im_file = exif_rotate(PILImage.open(im_name))
        # =============================================================================
        # INITIALIZE IMAGE
        # =============================================================================

        #print("You upload a file:")
        im_file = get_photo(btn_upload)
        # initialize Image object

        #im_file = PILImage.open(io.BytesIO(btn_upload.data[-1]))
        
        # display(im_file)
                
        # =============================================================================
        # DETECT PRICETAGS
        # =============================================================================
        try: 
            det_pricetags = yolo_price_model.detect([np.array(im_file)],  mode="return")

            # =============================================================================
            # CUT PRICETAGS
            # =============================================================================        

            # returns list of n centermost pricetags. To return only coordinates, use mode="return" 
            #cuts = get_centermost_detections(im_file, det_pricetags, n=1, mode="cut",bigger_box_by_pct=0.00)


            print(f"{len(det_pricetags['cls'])} nalezených cenovek, čtu cenovku nejblíže středu snímku.")
            # display(PILImage.fromarray(cuts[0]))

            # TODO: filter pricetag and read it
            sel_pricetags = get_centermost_detections(im_file,
                                                      det_pricetags,
                                                      n=1,
                                                      mode="cut",
                                                      bigger_box_by_pct=0.0
                                                      )
            # detected product name and price bounding boxes (from a single pricetag)
            det_product_name = yolo_product_name_model.detect([sel_pricetags[0]],  mode="return")

            # selected name and price bounding boxes (at most 3 assumed in a photo)
            # TODO: without using get_centermost_detec
            sel_pricetag_parts = get_centermost_detections(sel_pricetags[0],
                                                           det_product_name,
                                                           n=4,
                                                           mode="cut",
                                                           bigger_box_by_pct=0.0,
                                                           centerpoint=np.array([0,0]),
                                                           )


            # =============================================================================
            # READ PRICETAGS
            # =============================================================================   

            # saving cuts to be able to run Google API detection (custom load)
            cuts_path = save_cuts_for_detection(sel_pricetag_parts)

            # run Google API text detection and OCR
            cuts_ocr = [detect_text(path) for path in cuts_path]
            # cuts_drawn = [draw_ocr(ocr) for ocr in cuts_ocr]
            # display(cuts_drawn[0])
            # [os.remove(path) for path in cuts_path]

            product_name = cuts_ocr[0][1]
            x = cuts_ocr[1][1]

            # heuristic 1
            if len(x) == 4:
                # heuristic 2
                if re.match(heur_2_string, x[1]) and not re.match(heur_2_string, x[2]):
                    swap = x[1]
                    x[1] = x[2]
                    x[2] = swap

            # heuristic 2
            x[-1] = re.sub(r"K.$","Kč",x[-1])
            if re.match(r"Kč",x[-1]):
                del x[-1]

            # heuristic 3
            if len(x) < 4:
                # split off haller
                if re.match(heur_3_string, x[-1]):
                    x.append(x[-1][-2:])
                    x[-2] = x[-2][:-2]
               # elif re.match(heur_3_string, x[-2]) and re.match():

            #heuristic 5
            for i in range(1,len(x)):
                x[i] = re.sub(r"\D","",x[i])


            price = x 
            # print results 
            print(product_name[0])  
            print("Cena",",".join(price[1:]))

            display(PILImage.fromarray(sel_pricetag_parts[0]))
            display(PILImage.fromarray(sel_pricetag_parts[1]))
        except:
            print("Nedokážu přečíst obrázek, zkuste to prosím znovu.")
    return

In [24]:
btn_upload = widgets.FileUpload()

In [25]:
_ = interact(callback, value=btn_upload)

interactive(children=(FileUpload(value={}, description='Upload'), Output()), _dom_classes=('widget-interact',)…